Dialogue Summarisation with LLMs

Murray Shanahan

January 2025

In [8]:
import os
import sys
import json
from iconic_tools.langchain import InstructSonnet, InstructOpus3, InstructGPT4, InstructO1, InstructGeminiPro, InstructGeminiFlash, InstructGeminiFlash2,  InstructGPT35
from langchain_core.prompts import ChatPromptTemplate

In [9]:
# CONSTANTS AND INITIALISATION

PATH = os.path.abspath(os.getcwd())

# DIALOGUE_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGeminiFlash(temperature=1.0, max_tokens=3000)

DIALOGUE_MODEL = InstructGeminiFlash2(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructO1()

# DIALOGUE_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)

GAME = "act_1"

RED = "\033[91m"
GREEN = "\033[92m"
BLUE = "\033[94m"
YELLOW = "\033[93m"
WHITE = "\033[0m"

In [10]:
# UTILITIES


def load_prompt(filename):
    with open(PATH + f"/prompts/{filename}") as f:
        return f.read()


def load_dialogue(filename):
    with open(PATH + f"/{filename}") as f:
        return f.read()
    

def write_summary(summary, filename):
    with open(PATH + f"/temp/{filename}", "w") as f:
        f.write(summary)

In [11]:
# PROMPT TEMPLATES AND INSTRUCTION PROMPTS


summary_instruction_prefix = load_prompt("/summary_instruction_prefix.txt")

summary_preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
"""

summary_instruction_template = """
{preamble}
Here is the dialogue you need to work with:\n
{dialogue}
{instruction_suffix}
"""

summary_instruction_suffix = """
Give me a short paragraph summarising any information revealed by the player that might be relevant for later dialogues, for example personal or biographical information.\n
"""


merge_instruction_prefix = load_prompt("/merge_instruction_prefix.txt")

merge_preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
"""

merge_instruction_template = """
{preamble}
Here is the first summary:\n
{summary1} \n
Here is the second summary:\n
{summary2} \n
{instruction_suffix}
"""

merge_instruction_suffix = """
Give me a short paragraph summarising any biographical information revealed by the player or the NPCs that might be relevant for later dialogues.\n
"""


In [12]:
# GENERATING SUMMARIES


def prompt_llm(prompt, model):
    # print(prompt)
    # print()
    prompt = ChatPromptTemplate.from_template(template=prompt)
    chain = prompt | model
    return chain


def summarise_dialogue(dialogue):
    """Generates dialogue summary for a mini-scene.
    """

    dialogue_model = DIALOGUE_MODEL
 
    back_story = load_prompt(GAME + "/back_story.txt")
        
    preamble = summary_preamble_template.format(
        instruction_prefix=summary_instruction_prefix,
        back_story=back_story)
            
    prompt = summary_instruction_template.format(
        preamble=preamble, dialogue=dialogue,
        instruction_suffix=summary_instruction_suffix)
    chain = prompt_llm(prompt, dialogue_model)
    response = chain.invoke({})

    summary = response.strip()
    
    return (summary)


def merge_summaries(summary1, summary2):
    """Merges two dialogue summaries, removing redundant information.
    """

    dialogue_model = DIALOGUE_MODEL
 
    back_story = load_prompt(GAME + "/back_story.txt")
        
    preamble = merge_preamble_template.format(
        instruction_prefix=merge_instruction_prefix,
        back_story=back_story)
            
    prompt = merge_instruction_template.format(
        preamble=preamble,
        summary1=summary1, summary2=summary2,
        instruction_suffix=merge_instruction_suffix)
    chain = prompt_llm(prompt, dialogue_model)
    response = chain.invoke({})

    summary = response.strip()
    
    return (summary)


In [13]:
# GENERATE A SUMMARY


dialogue = load_dialogue("dialogue_samples/space_walk_samples/sample_dialogue1.txt")

summary1 = summarise_dialogue(dialogue)

write_summary(summary1, "dialogue_summary.txt")

print('SUMMARY 1')
print()
print(summary1 + '\n\n')

SUMMARY 1

The player is a botanist by training, now on their first space mission to Terra Nova. They were inspired to join the mission by a love of science fiction and a desire to contribute to long-term survival through plant-based nutrition. They grew up in London, and their mother, from whom they inherited their love of plants, currently lives in New York City. Their father died when they were young.




In [14]:
# GENERATE ANOTHER SUMMARY AND MERGE THE TWO

dialogue = load_dialogue("dialogue_samples/space_walk_samples/sample_dialogue2.txt")

summary2 = summarise_dialogue(dialogue)

print('SUMMARY 2')
print()
print(summary2 + '\n\n')

merged_summary = merge_summaries(summary1, summary2)

print('MERGED SUMMARY')
print()
print(merged_summary)

SUMMARY 2

The player was previously involved in a mission involving commercial secrets. Before being put into suspended animation, the player lived in London and often visited the science museum, especially the robot floor.


MERGED SUMMARY

The player is a botanist on their first space mission to Terra Nova, motivated by a love of science fiction and a desire to contribute to long-term survival. They previously worked on a mission involving commercial secrets. They grew up in London and often visited the science museum, particularly the robot floor. Their mother, who instilled in them a love of plants, currently lives in New York City. Their father died when they were young.
